In [15]:
import numpy as np
import pandas as pd
import torch as th
import os
from collections import Counter
import h5py

from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import classification_report

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import random

In [9]:
new_simple_act = ['unlabeled',
 'driving bike',
 'having dinner',
 'having lunch',
 'using the toilet',
 'queuing in line',
 'walking',
 'walking freely',
 'washing hands',
 'sitting',
 'standing',
 'having a coffee',
 'lying',
 'using computer',
 'desk activities',
 'carrying something']
new_complex_act = ['unlabeled',
 'driving car',
 'personal hygiene',
 'sitting  having a coffee',
 'sitting  desk activities',
 'walking while carrying something',
 'picking up mensa food',
 'lying  using computer',
 'discussing at whiteboard',
 'dinner activities',
 'commuting',
 'lunch routine',
 'office work']

In [3]:
data_df = pd.read_csv("ubicomp08_new_complex_one_hot_data_all_days_filtered_labels.csv")

data_df.sort_values(by = ['timestamp'], inplace = True)
data_df.timestamp = pd.to_datetime(data_df.timestamp,unit='ns',  origin = 'unix')
data_df.set_index(data_df.timestamp, drop = False, inplace= True)
data_df = data_df[~data_df.index.duplicated(keep='first')]

In [4]:
data_df.columns

Index(['mean_poc_x', 'mean_poc_y', 'mean_poc_z', 'var_poc_x', 'var_poc_y',
       'var_poc_z', 'mean_wri_x', 'mean_wri_y', 'mean_wri_z', 'var_wri_x',
       'var_wri_y', 'var_wri_z', 'timestamp', 'unlabeled', 'driving bike',
       'having dinner', 'having lunch', 'using the toilet', 'queuing in line',
       'walking', 'walking freely', 'washing hands', 'sitting', 'standing',
       'having a coffee', 'lying', 'using computer', 'desk activities',
       'carrying something', 'unlabeled.1', 'driving car', 'personal hygiene',
       'sitting  having a coffee', 'sitting  desk activities',
       'walking while carrying something', 'picking up mensa food',
       'lying  using computer', 'discussing at whiteboard',
       'dinner activities', 'commuting', 'lunch routine', 'office work',
       'complex_category', 'day'],
      dtype='object')

In [30]:
data_df["activity"] = np.argmax(data_df.iloc[:, 13:29].values, axis = 1)

In [31]:
data_df.to_csv("ubicomp08_new_complex_one_hot_data_all_days_filtered_labels_LDA.csv", index = False)

In [6]:
one_hot_enc = OneHotEncoder()

In [7]:
test_days_list = [1,2,3,4,5,6,7]
random.shuffle(test_days_list)
test_days_list

[4, 7, 3, 6, 1, 2, 5]

In [25]:
sa_acc_list, sa_f1_list, sa_prec_list, sa_rec_list, sa_ba_list, ca_acc_list, ca_f1_list, ca_prec_list, ca_rec_list, ca_ba_list = [], [], [], [], [], [], [], [], [], []

for fold_idx in range(7):
    
    x_train, y_simple_train, y_complex_train, x_test, y_simple_test, y_complex_test = [], [], [], [], [], []

    test_df = data_df[data_df.day == test_days_list[fold_idx]]

    train_df = data_df[data_df.day != test_days_list[fold_idx]]

    x_train = train_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values
    y_simple_train = train_df.iloc[:, 13:29].values
    y_complex_train = train_df.iloc[:, 29:42].values
    x_test = test_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values
    y_simple_test = test_df.iloc[:, 13:29].values
    y_complex_test = test_df.iloc[:, 29:42].values
    
    
    num_classes = len(new_simple_act)
    clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))
    clf.fit(x_train, y_simple_train)
    predictions = clf.predict(x_test)
    print("Fold : {}".format(fold_idx))
    print("Simple :")
    print(classification_report(y_simple_test, predictions, target_names = new_simple_act))
    accuracy = accuracy_score(y_simple_test, predictions)
    print("Accuracy: %.4f" % (accuracy))
    sa_acc_list.append(accuracy)
    f1 = f1_score(y_simple_test, predictions, average='macro')
    print("F1: %.4f" % (f1))
    sa_f1_list.append(f1)
    prec = precision_score(y_simple_test, predictions, average='macro')
    print("Precision: %.4f" % (prec))
    sa_prec_list.append(prec)
    recall = recall_score(y_simple_test, predictions, average='macro')
    print("Recall: %.4f" % (recall))
    sa_rec_list.append(recall)
    ba = balanced_accuracy_score(np.argmax(y_simple_test, axis = 1), np.argmax(predictions, axis = 1))
    print("BA: %.4f" % (ba))
    sa_ba_list.append(ba)
    
    num_classes = len(new_complex_act)
    clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))
    clf.fit(x_train, y_complex_train)
    predictions = clf.predict(x_test)
    print("Complex :")
    print(classification_report(y_complex_test, predictions, target_names = new_complex_act))
    accuracy = accuracy_score(y_complex_test, predictions)
    print("Accuracy: %.4f" % (accuracy))
    ca_acc_list.append(accuracy)
    f1 = f1_score(y_complex_test, predictions, average='macro')
    print("F1: %.4f" % (f1))
    ca_f1_list.append(f1)
    prec = precision_score(y_complex_test, predictions, average='macro')
    print("Precision: %.4f" % (prec))
    ca_prec_list.append(prec)
    recall = recall_score(y_complex_test, predictions, average='macro')
    print("Recall: %.4f" % (recall ))
    ca_rec_list.append(recall)
    ba = balanced_accuracy_score(np.argmax(y_simple_test, axis = 1), np.argmax(predictions, axis = 1))
    print("BA: %.4f" % (ba))
    ca_ba_list.append(ba)
print("Simple Acc: {}".format(np.mean(sa_acc_list)))
print("Simple F1: {}".format(np.mean(sa_f1_list)))
print("Simple Prec: {}".format(np.mean(sa_prec_list)))
print("Simple Rec: {}".format(np.mean(sa_rec_list)))
print("Simple BA: {}".format(np.mean(sa_ba_list)))

print("Complex Acc: {}".format(np.mean(ca_acc_list)))
print("Complex F1: {}".format(np.mean(ca_f1_list)))
print("Complex Prec: {}".format(np.mean(ca_prec_list)))
print("Complex Rec: {}".format(np.mean(ca_rec_list)))
print("Complex BA: {}".format(np.mean(ca_ba_list)))

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/multiclass.py:74: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/multiclass.py:74: UserWarning: Label not 10 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


Fold : 0
Simple :
                    precision    recall  f1-score   support

         unlabeled       0.90      0.87      0.88     17579
      driving bike       0.00      0.00      0.00         0
     having dinner       0.14      0.14      0.14      1657
      having lunch       0.44      0.28      0.34       746
  using the toilet       0.17      0.01      0.01       129
   queuing in line       0.00      0.00      0.00       135
           walking       0.34      0.11      0.17       663
    walking freely       0.66      0.80      0.73       959
     washing hands       0.00      0.00      0.00         0
           sitting       0.87      0.67      0.75     21189
          standing       0.00      0.00      0.00         0
   having a coffee       0.00      0.00      0.00       307
             lying       0.00      0.00      0.00         0
    using computer       0.00      0.00      0.00         0
   desk activities       0.88      0.63      0.73     19548
carrying something   

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ho

F1: 0.2376
Precision: 0.2805
Recall: 0.2208


/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


...

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


BA: 0.3973
Complex :
                                  precision    recall  f1-score   support

                       unlabeled       0.35      0.23      0.28      8386
                     driving car       0.97      0.79      0.87      1103
                personal hygiene       0.00      0.00      0.00         0
        sitting  having a coffee       0.00      0.00      0.00       220
        sitting  desk activities       0.79      0.88      0.84     24903
walking while carrying something       0.10      0.08      0.09       106
           picking up mensa food       0.44      0.01      0.03       311
           lying  using computer       0.00      0.00      0.00         0
        discussing at whiteboard       0.00      0.00      0.00         0
               dinner activities       0.08      0.00      0.01      1905
                       commuting       0.60      0.09      0.16       917
                   lunch routine       0.32      0.23      0.27      1892
                

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.2956
Recall: 0.1808
BA: 0.0708
Simple Acc: 0.7204664864647036
Simple F1: 0.3119626340044849
Simple Prec: 0.37721983283479915
Simple Rec: 0.28984713293217074
Simple BA: 0.4122453389320291
Complex Acc: 0.6055037892426655
Complex F1: 0.23007331478791762
Complex Prec: 0.35970858414919676
Complex Rec: 0.20117753366556482
Complex BA: 0.08567633363899459


/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [23]:
balanced_accuracy_score(np.argmax(y_simple_test, axis = 1), np.argmax(predictions, axis = 1))

0.3531547386464552

In [11]:
"predictions

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [32]:
data_df = pd.read_csv("/media/kavin/silo/ghost-storage/Research/WASH/complex_activities/ADL-TM-master/data/ubicomp/LDA_T20s40/processed.csv")

data_df.sort_values(by = ['timestamp'], inplace = True)
data_df.timestamp = pd.to_datetime(data_df.timestamp,unit='ns',  origin = 'unix')
data_df.set_index(data_df.timestamp, drop = False, inplace= True)
data_df = data_df[~data_df.index.duplicated(keep='first')]

In [35]:
data_df.columns[47]

'topic'

In [36]:
sa_acc_list, sa_f1_list, sa_prec_list, sa_rec_list, sa_ba_list, ca_acc_list, ca_f1_list, ca_prec_list, ca_rec_list, ca_ba_list = [], [], [], [], [], [], [], [], [], []

for fold_idx in range(7):
    
    x_train, y_simple_train, y_complex_train, x_test, y_simple_test, y_complex_test = [], [], [], [], [], []

    test_df = data_df[data_df.day == test_days_list[fold_idx]]

    train_df = data_df[data_df.day != test_days_list[fold_idx]]

    x_train = train_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 46, 47]].values
    y_simple_train = train_df.iloc[:, 13:29].values
    y_complex_train = train_df.iloc[:, 29:42].values
    x_test = test_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 46, 47]].values
    y_simple_test = test_df.iloc[:, 13:29].values
    y_complex_test = test_df.iloc[:, 29:42].values
    
    
    num_classes = len(new_simple_act)
    clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))
    clf.fit(x_train, y_simple_train)
    predictions = clf.predict(x_test)
    print("Fold : {}".format(fold_idx))
    print("Simple :")
    print(classification_report(y_simple_test, predictions, target_names = new_simple_act))
    accuracy = accuracy_score(y_simple_test, predictions)
    print("Accuracy: %.4f" % (accuracy))
    sa_acc_list.append(accuracy)
    f1 = f1_score(y_simple_test, predictions, average='macro')
    print("F1: %.4f" % (f1))
    sa_f1_list.append(f1)
    prec = precision_score(y_simple_test, predictions, average='macro')
    print("Precision: %.4f" % (prec))
    sa_prec_list.append(prec)
    recall = recall_score(y_simple_test, predictions, average='macro')
    print("Recall: %.4f" % (recall))
    sa_rec_list.append(recall)
    ba = balanced_accuracy_score(np.argmax(y_simple_test, axis = 1), np.argmax(predictions, axis = 1))
    print("BA: %.4f" % (ba))
    sa_ba_list.append(ba)
    
    num_classes = len(new_complex_act)
    clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))
    clf.fit(x_train, y_complex_train)
    predictions = clf.predict(x_test)
    print("Complex :")
    print(classification_report(y_complex_test, predictions, target_names = new_complex_act))
    accuracy = accuracy_score(y_complex_test, predictions)
    print("Accuracy: %.4f" % (accuracy))
    ca_acc_list.append(accuracy)
    f1 = f1_score(y_complex_test, predictions, average='macro')
    print("F1: %.4f" % (f1))
    ca_f1_list.append(f1)
    prec = precision_score(y_complex_test, predictions, average='macro')
    print("Precision: %.4f" % (prec))
    ca_prec_list.append(prec)
    recall = recall_score(y_complex_test, predictions, average='macro')
    print("Recall: %.4f" % (recall ))
    ca_rec_list.append(recall)
    ba = balanced_accuracy_score(np.argmax(y_simple_test, axis = 1), np.argmax(predictions, axis = 1))
    print("BA: %.4f" % (ba))
    ca_ba_list.append(ba)
print("Simple Acc: {}".format(np.mean(sa_acc_list)))
print("Simple F1: {}".format(np.mean(sa_f1_list)))
print("Simple Prec: {}".format(np.mean(sa_prec_list)))
print("Simple Rec: {}".format(np.mean(sa_rec_list)))
print("Simple BA: {}".format(np.mean(sa_ba_list)))

print("Complex Acc: {}".format(np.mean(ca_acc_list)))
print("Complex F1: {}".format(np.mean(ca_f1_list)))
print("Complex Prec: {}".format(np.mean(ca_prec_list)))
print("Complex Rec: {}".format(np.mean(ca_rec_list)))
print("Complex BA: {}".format(np.mean(ca_ba_list)))

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/multiclass.py:74: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/multiclass.py:74: UserWarning: Label not 10 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


Fold : 0
Simple :
                    precision    recall  f1-score   support

         unlabeled       0.90      0.87      0.88     17579
      driving bike       0.00      0.00      0.00         0
     having dinner       0.14      0.14      0.14      1657
      having lunch       0.44      0.28      0.34       746
  using the toilet       0.17      0.01      0.01       129
   queuing in line       0.00      0.00      0.00       135
           walking       0.34      0.11      0.17       663
    walking freely       0.66      0.80      0.73       959
     washing hands       0.00      0.00      0.00         0
           sitting       0.87      0.67      0.75     21189
          standing       0.00      0.00      0.00         0
   having a coffee       0.00      0.00      0.00       307
             lying       0.00      0.00      0.00         0
    using computer       0.00      0.00      0.00         0
   desk activities       0.88      0.63      0.73     19548
carrying something   

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ho

F1: 0.2376
Precision: 0.2805
Recall: 0.2208


/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


...

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


BA: 0.3973
Complex :
                                  precision    recall  f1-score   support

                       unlabeled       0.35      0.23      0.28      8386
                     driving car       0.97      0.79      0.87      1103
                personal hygiene       0.00      0.00      0.00         0
        sitting  having a coffee       0.00      0.00      0.00       220
        sitting  desk activities       0.79      0.88      0.84     24903
walking while carrying something       0.10      0.08      0.09       106
           picking up mensa food       0.44      0.01      0.03       311
           lying  using computer       0.00      0.00      0.00         0
        discussing at whiteboard       0.00      0.00      0.00         0
               dinner activities       0.08      0.00      0.01      1905
                       commuting       0.60      0.09      0.16       917
                   lunch routine       0.32      0.23      0.27      1892
                

/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.2956
Recall: 0.1808
BA: 0.0708
Simple Acc: 0.7204664864647036
Simple F1: 0.3119626340044849
Simple Prec: 0.37721983283479915
Simple Rec: 0.28984713293217074
Simple BA: 0.4122453389320291
Complex Acc: 0.6055037892426655
Complex F1: 0.23007331478791762
Complex Prec: 0.35970858414919676
Complex Rec: 0.20117753366556482
Complex BA: 0.08567633363899459


/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kavin/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
